<a href="https://colab.research.google.com/github/VellummyilumVinoth/Aspect_based_sentimental_analysis/blob/main/recommendation_system_for_old_and_new_user.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [58]:

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [85]:
# Step 1: Load the dataset
import pandas as pd
dataset = pd.read_csv('/content/drive/MyDrive/zidan/aspect_sentiments.csv')

In [86]:
columns_to_drop = ['Price', 'Packaging', 'Delivery Cost', 'Quality', 'Overall Sentiment']
dataset.drop(columns_to_drop, axis=1, inplace=True)

In [87]:
dataset.head(10)

,Product_name,Review,Overall Sentiment Score
0,"Salate Cordless Electric Spin Scrubber, Electr...",I absolutely love this wireless electric spin ...,0
1,"Salate Cordless Electric Spin Scrubber, Electr...",i used this to clean my bathroom and this work...,1
2,"Salate Cordless Electric Spin Scrubber, Electr...",This electric brush makes cleaning so much eas...,2
3,"Salate Cordless Electric Spin Scrubber, Electr...",This Electric spin scrubber is perfect! Use it...,0
4,"Salate Cordless Electric Spin Scrubber, Electr...",I don’t like cleaning but this machine help me...,0
5,"Salate Cordless Electric Spin Scrubber, Electr...",It has really made my life so much easy! Its v...,0
6,"Salate Cordless Electric Spin Scrubber, Electr...",So far I've used this powered scrubber a coupl...,1
7,"Salate Cordless Electric Spin Scrubber, Electr...",I did not like to clean the kitchen. This mach...,0
8,"Charmin Ultra Strong Clean Touch Toilet Paper,...","Ladies and gentlemen, get ready for a review t...",0
9,"Charmin Ultra Strong Clean Touch Toilet Paper,...","Oh, Ultra Strong Toilet Paper, how you do\nPro...",0


In [88]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import random

# Generate random reviewer names
def generate_reviewer_names(num_reviewers):
    first_names = ["John", "Jane", "Michael", "Emily", "David", "Olivia", "Daniel", "Sophia", "Matthew", "Emma", "Zidan", "Thinesh", "Albert", "Vinoth", "Saini", "Karthi"]
    last_names = ["Smith", "Johnson", "Brown", "Taylor", "Anderson", "Williams", "Jones", "Davis", "Miller", "Wilson", "Maryam", "Soosai", "Christopher","Vithujan", "Kumar", "Bathri"]
    reviewer_names = []
    for _ in range(num_reviewers):
        first_name = random.choice(first_names)
        last_name = random.choice(last_names)
        reviewer_names.append(f"{last_name} {first_name}")
    return reviewer_names

# Number of reviewers to generate
num_reviewers = 500

# Generate reviewer information
reviewer_names = generate_reviewer_names(num_reviewers)
reviewer_ages = np.random.randint(low=18, high=76, size=num_reviewers)
reviewer_genders = np.random.choice(["Male", "Female"], size=num_reviewers)
reviewer_areas = np.random.choice(["Central", "Eastern", "Northern", "North Central", "North Western", "Sabaragamuwa", "Southern", "Uva", "Western"], size=num_reviewers)

# Create a dictionary to map reviewer names with their respective age, gender, and area
reviewer_info = {name: (age, gender, area) for name, age, gender, area in zip(reviewer_names, reviewer_ages, reviewer_genders, reviewer_areas)}

# Assign random reviewer names to each row in the dataset
dataset['reviewer_name'] = random.choices(reviewer_names, k=len(dataset))

# Assign reviewer age, gender, and area based on the reviewer name
dataset['age'], dataset['gender'], dataset['area'] = zip(*[reviewer_info[name] for name in dataset['reviewer_name']])

# Extract the features
reviews = dataset["Review"]
product_names = dataset["Product_name"]
overall_sentiment_score = dataset["Overall Sentiment Score"]
reviewer_name = dataset["reviewer_name"]
age = dataset["age"]
gender = dataset["gender"]
area = dataset["area"]

In [89]:
dataset

,Product_name,Review,Overall Sentiment Score,reviewer_name,age,gender,area
0,"Salate Cordless Electric Spin Scrubber, Electr...",I absolutely love this wireless electric spin ...,0,Anderson Karthi,32,Male,Southern
1,"Salate Cordless Electric Spin Scrubber, Electr...",i used this to clean my bathroom and this work...,1,Anderson Saini,34,Female,Southern
2,"Salate Cordless Electric Spin Scrubber, Electr...",This electric brush makes cleaning so much eas...,2,Christopher Sophia,69,Male,Eastern
3,"Salate Cordless Electric Spin Scrubber, Electr...",This Electric spin scrubber is perfect! Use it...,0,Bathri Olivia,58,Female,Northern
4,"Salate Cordless Electric Spin Scrubber, Electr...",I don’t like cleaning but this machine help me...,0,Anderson Emma,25,Female,Sabaragamuwa
...,...,...,...,...,...,...,...
11052,2 Pack Pumice Stone for Toilet Cleaning with E...,Work great,0,Williams David,26,Male,Western
11053,2 Pack Pumice Stone for Toilet Cleaning with E...,The tool works fairly well for removing hard s...,4,Soosai Zidan,32,Male,Eastern
11054,2 Pack Pumice Stone for Toilet Cleaning with E...,Long handle so you don’t have to put your hand...,1,Williams Emily,35,Female,Central
11055,2 Pack Pumice Stone for Toilet Cleaning with E...,It works very nicely on these Arizona hard wat...,1,Vithujan Jane,28,Male,Eastern


## Content based filtering for new user using age, gender and area

In [90]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Create a TF-IDF vectorizer
vectorizer = TfidfVectorizer()

# Transform the reviews into TF-IDF vectors
review_vectors = vectorizer.fit_transform(reviews)

# Calculate the cosine similarity between all pairs of products
product_similarity = cosine_similarity(review_vectors)

# Create a function to recommend products to a user
def recommend_products(user_age, user_gender, user_area):
    # Validate age input
    if user_age <= 18 or user_age >= 75:
        raise ValueError("Age should be between 18 and 75.")
        
    # Validate gender input
    valid_genders = ["Male", "Female", "Other"]
    if user_gender not in valid_genders:
        raise ValueError("Invalid gender. Please enter 'Male', 'Female', or 'Other'.")
        
    # Validate area input
    valid_areas = ["Central", "Eastern", "Northern", "North Central", "North Western", "Sabaragamuwa", "Southern", "Uva", "Western"]
    if user_area not in valid_areas:
        raise ValueError("Invalid area. Please enter a valid province name.")

    # Filter products that match the user's age, gender, and area
    filtered_indices = np.where((age == user_age) & (gender == user_gender) & (area == user_area))[0]

    # Calculate the similarity scores for the filtered products
    similarity_scores = np.mean(product_similarity[filtered_indices], axis=0)

    # Sort the products by their similarity to the user
    similar_products = np.argsort(similarity_scores)[::-1]

    # Initialize set to keep track of recommended product names
    recommended_set = set()

    # Return the top 10 unique products with their scores
    recommendations = []
    for rank, product in enumerate(similar_products):
        if len(recommendations) >= 10:
            break
        product_name = product_names[product]
        if product_name not in recommended_set:
            score = similarity_scores[product]
            recommendations.append((rank + 1, product_name, score))
            recommended_set.add(product_name)
    return recommendations

# Recommend products to a user
user_age = 34
user_gender = "Female"
user_area = "Western"

try:
    recommended_products = recommend_products(user_age, user_gender, user_area)
    # Print the recommended products with rank numbers and scores
    for rank, product in recommended_products:
        print(f"Rank {rank}: {product}")
except ValueError as e:
    print("Error:", str(e))

Error: too many values to unpack (expected 2)


In [91]:
dataset

,Product_name,Review,Overall Sentiment Score,reviewer_name,age,gender,area
0,"Salate Cordless Electric Spin Scrubber, Electr...",I absolutely love this wireless electric spin ...,0,Anderson Karthi,32,Male,Southern
1,"Salate Cordless Electric Spin Scrubber, Electr...",i used this to clean my bathroom and this work...,1,Anderson Saini,34,Female,Southern
2,"Salate Cordless Electric Spin Scrubber, Electr...",This electric brush makes cleaning so much eas...,2,Christopher Sophia,69,Male,Eastern
3,"Salate Cordless Electric Spin Scrubber, Electr...",This Electric spin scrubber is perfect! Use it...,0,Bathri Olivia,58,Female,Northern
4,"Salate Cordless Electric Spin Scrubber, Electr...",I don’t like cleaning but this machine help me...,0,Anderson Emma,25,Female,Sabaragamuwa
...,...,...,...,...,...,...,...
11052,2 Pack Pumice Stone for Toilet Cleaning with E...,Work great,0,Williams David,26,Male,Western
11053,2 Pack Pumice Stone for Toilet Cleaning with E...,The tool works fairly well for removing hard s...,4,Soosai Zidan,32,Male,Eastern
11054,2 Pack Pumice Stone for Toilet Cleaning with E...,Long handle so you don’t have to put your hand...,1,Williams Emily,35,Female,Central
11055,2 Pack Pumice Stone for Toilet Cleaning with E...,It works very nicely on these Arizona hard wat...,1,Vithujan Jane,28,Male,Eastern


## Collaborative filtering for old user using their sentimental score for the reviews by them

In [92]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Filter the data for the old user
old_user = 'Johnson Sophia'
old_user_data = dataset[dataset['reviewer_name'] == old_user]

# Filter the data for products with sentiment score less than 2
filtered_data = dataset[dataset['Overall Sentiment Score'] < 2].copy()

# Create a TF-IDF vectorizer
vectorizer = TfidfVectorizer()

# Apply TF-IDF vectorization to the review text
review_vectors = vectorizer.fit_transform(filtered_data['Review'])

# Calculate cosine similarity between the old user's reviews and the filtered products
cosine_similarities = cosine_similarity(vectorizer.transform(old_user_data['Review']), review_vectors)

# Get the average sentiment score for each product
filtered_data['Average_Sentiment'] = filtered_data.groupby('Product_name')['Overall Sentiment Score'].transform('mean')

# Combine the average sentiment scores with the cosine similarities
filtered_data['Score'] = cosine_similarities[0] * filtered_data['Average_Sentiment']

# Sort the data based on the score
sorted_data = filtered_data.sort_values('Score', ascending=False)

# Add ranking numbers to the sorted data
sorted_data['Rank'] = range(1, len(sorted_data) + 1)

# Get the top 10 recommended products with ranking numbers
top_10_products = sorted_data[['Rank', 'Product_name', 'Score']].head(10)

print("Top 10 recommended products for", old_user, "with their past reviews:")
print(top_10_products)


Top 10 recommended products for Johnson Sophia with their past reviews:
      Rank                                       Product_name     Score
7611     1  Turbo Bee 500Pack 9 OZ Clear Plastic Cups,Cold...  0.335189
7514     2  [400 Pack] Flexible Disposable Plastic Drinkin...  0.312912
6994     3  Duracell 376/377 Silver Oxide Button Battery, ...  0.301931
4846     4  100% Compostable 9 Inch Heavy-Duty Paper Plate...  0.293290
8886     5              SOLO Plastic Forks, White, 500 Ct Tej  0.283708
4308     6  Kleenex 2-Ply White Tissues, A4016, 210 Count ...  0.279324
5684     7  Lepro AAA Batteries 24 Pack, 1200mAh Triple A ...  0.265976
4971     8  Charmin Ultra Soft Toilet Paper, 18 Super Mega...  0.232729
768      9  Scotch-Brite Zero Scratch Non-Scratch Scrub Sp...  0.232535
7953    10  Tork Matic Paper Hand Towel Roll White H1, Uni...  0.231434


In [93]:
john_data = dataset[dataset['reviewer_name'] == 'Johnson Sophia']

# Print the data for reviewer name "Johnson Sophia"
john_data

,Product_name,Review,Overall Sentiment Score,reviewer_name,age,gender,area
246,"Scotch-Brite Non-Scratch Scrub Sponges, Sponge...","we use these sponges in the Kitchen, good prod...",0,Johnson Sophia,71,Male,Central
588,Energizer Alkaline Power AAA Batteries (32 Pac...,"While not all batteries are created equal, in ...",0,Johnson Sophia,71,Male,Central
710,"Salate Cordless Electric Spin Scrubber, Electr...",I don’t like cleaning but this machine help me...,0,Johnson Sophia,71,Male,Central
839,"Hefty Recycling Trash Bags, Blue, 13 Gallon, 6...",I thought these would be much better than not ...,2,Johnson Sophia,71,Male,Central
914,"Charmin Flushable Wipes, 4 packs, 40 Wipes Per...",Don’t waste your money they’re flimsy they tea...,4,Johnson Sophia,71,Male,Central
...,...,...,...,...,...,...,...
10276,"Bug Zapper Indoor & Outdoor, 2 in 1 Mosquito Z...",The product work as expected. You can use it a...,0,Johnson Sophia,71,Male,Central
10461,HINSOCHA 3 Pairs Rubber Cleaning Gloves Latex ...,"These gloves are pretty long, they cover the e...",4,Johnson Sophia,71,Male,Central
10484,"Vinda 3-Ply Ultra Care Pocket Tissues, Travel ...",Soft and absorbent,2,Johnson Sophia,71,Male,Central
10511,AmazonCommercial FSC Certified 2-Ply White Ada...,thin but as expected for the price,2,Johnson Sophia,71,Male,Central
